## Вопрос ответный бот 

Для создания вопрос-ответных ботов на какую-то конкретную тему используют подход, называемый **Retrival-Augmented Generation**. Бот опирается на какую-то коллекцию текстов - базу знаний. Когда пользователь задаёт вопрос, ищутся тексты, похожие на его вопрос - и даются в качестве входного текста модели GPT.

![](images/hp_rag.svg)

Для создания таких ботов часто используется библиотека [LangChain](https://python.langchain.com). Для начала, установим `langchain` и сопутствующие библиотеки.

In [ ]:
%pip install nltk==3.9.1 telebot==0.0.5 langchain==0.2.1 telebot sentence_transformers==3.2.1 langchain_community==0.2.4 langchain_chroma==0.1.4 unstructured yandex_chain==0.0.9 yandexcloud

## Википедия Гарри Поттера

В нашем примере мы будет строить бота на основе [Harry Potter Fandom Wiki](https://harrypotter.fandom.com/). Тексты из этой вики уже находятся в репозитории, их надо лишь разархивировать:


In [ ]:
!7za x data/harry_ctext.zip

У нас в проекте появилась директория `content`, в которой лежат все текста. Вот как выглядит отдельно взятый текст:

In [ ]:
with open('content/ctext/Белые искры.txt') as f:
    print(f.read())

Выполним несколько подготовительных действий...

In [ ]:
import nltk
nltk.download('punkt')

### Разбиваем текст на фрагменты

> **ВНИМАНИЕ!!!** Ниже мы будем строить базу знаний для бота на основе текстов о Гарри Поттере. Поскольку построение базы знаний - времязатратное мероприятие, **вы можете пропустить** (не выполнять) **следующие несколько ячеек**, если просто хотите посмотреть, как работает бот. Если вы решите создавать своего бота на основе своей текстовой базы знаний, то нужно будет выполнять ячейки ниже, чтобы проиндексировать все тексты. 

Для поиска нам нужно будет выбирать достаточно небольшие фрагменты текста. Поэтому, если вдруг в базе есть большие тексты, из необходимо разбить на кусочки. Вот как это сделать:

In [ ]:
import langchain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader

chunk_size = 2048
chunk_overlap = 100
source_dir = "content/ctext"

loader = DirectoryLoader(
    source_dir, glob="*.txt", show_progress=True, recursive=True
)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
fragments = splitter.create_documents([x.page_content for x in loader.load()])
print(f"Количество кусочков = {len(fragments)}")

### Сохраняем фрагменты в базу данных

Для поиска нужных фрагментов мы будем использовать хитрый подход - **эмбеддинги**. Каждому тексту будет сопоставляться некоторый вектор (набор числел), в соответствии с его смыслом. Мы дальше будем просто искать похожие вектора на наш запрос.

Вот как можно вычислить такой вектор для текста:

> Не забудьте добавить секрет **api_key** и **folder_id** в свой проект DataSphere!


In [ ]:
from yandex_chain import YandexEmbeddings
import os

api_key = os.environ["api_key"]
folder_id = os.environ["folder_id"]

embeddings = YandexEmbeddings(folder_id=folder_id, api_key=api_key, sleep_interval=0.1)
vec = embeddings.embed_query("Hello, world!")
len(vec)

Теперь нам нужно проиндексировать все кусочки текстов

> **ВНИМАНИЕ**: Этот код выполняется долго (**20 минут**)! Выполняйте его только в том случае, если вы поменяете содержимое базы знаний, чтобы сделать своего бота!

In [ ]:
from langchain_chroma import Chroma

db_dir = "content"

db = Chroma(persist_directory=db_dir, embedding_function=embeddings)
db.add_documents(fragments)

> **ВНИМАНИЕ!!!** Чтобы просто посмотреть, как работает пример с Гарри Поттером, воспользуемся уже созданной ранее векторной базой данных и просто её разархивируем. Если вы делаете своего бота, то эту ячейку выполнять **не нужно**, чтоюы не стереть созданную ранее базу знаний.

In [ ]:
!7za x data/cdb.zip

Подключимся к этой базе:

In [ ]:
from langchain_chroma import Chroma

db_dir = "content"

vec_store = Chroma(persist_directory=db_dir, embedding_function=embeddings)

Теперь посмотрим, насколько хорошо находятся фрагменты текста по какому-то запросу:

In [ ]:
q = "Какое заклинание помогает левитировать?"

retriever = vec_store.as_retriever(search_kwargs={"k": 5})
res = retriever.invoke(q)
for x in res:
    print("-" * 40)
    print(x.page_content)

### Подключаемся к Yandex GPT

Для формирования целостного ответа на вопрос пользователей нам потребуется обработать найденные фрагменты текста с помощью Yandex GPT. Мы уже учились с вами ранее (в файле **Кусочки**) работать с YandexGPT, здесь повторяем этот код.

In [ ]:
from yandex_chain import YandexLLM,YandexGPTModel

instructions = """
Ты - дружелюбный чат-бот, отвечающий на вопросы про вселенную Гарри Поттера
"""

llm = YandexLLM(folder_id=folder_id,api_key=api_key, instruction_text=instructions, model=YandexGPTModel.Pro)

llm.invoke(q)

В данном примере мы пока что никак не использовали наши текстовые документы.

### Собираем Retrieval-Augmented Generation

Пришла пора соединить всё вместе и научить бота отвечать на вопросы, подглядывая в нашу текстовую базу знаний . Для этого используем механизм цепочек (*chains*):

In [ ]:
import langchain 
import langchain.prompts
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


prompt = """
Ты - бот, умеющий разноваривать про вселенную Гарри Поттера. Пожалуйста, посмотри на 
текст ниже и ответь на вопрос, используя информацию из этого текста. Не надо писать про текст,
пиши просто ответ, но достаточно подробный.
Текст:
-----
{context}
-----
Вопрос:
{question}"""

prompt = langchain.prompts.PromptTemplate(
    template=prompt, input_variables=["context", "question"]
)

def join_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Создаём цепочку
chain = (
    {"context": retriever | join_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

answer = chain.invoke(q)
print(answer)

### Делаем телеграм-бота

Теперь сделаем телеграм-бота для ответа на вопросы про вселенную Гарри Поттера. Как делать телеграм-бота мы уже знаем из примеров в файле **Кирпичики**. Сначала надо создать телеграм-бота в телеграме (с помощью **botfather**), и запомнить его секретный ключ в секрете DataSphere `tg_token`.

In [ ]:
import io,os
import telebot
from PIL import Image

telegram_token = os.environ['tg_token']

bot = telebot.TeleBot(telegram_token)

# Обработчик команды /start
@bot.message_handler(commands=['start'])
def start(message):
    # Отправляем приветственное сообщение
    bot.send_message(message.chat.id,
                     'Привет, я бот, который знает всё про вселенную Гарри Поттера. Спрашивай!')

# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    answer = chain.invoke(message.text)
    bot.send_message(message.chat.id, answer)
    
# Запуск бота
print("Бот готов к работе")
bot.polling(none_stop=True)

### Делаем веб-интерфейс для чата

Мы можем сделать простой веб-интерфейс для чата с помощью Gradio.

In [ ]:
import gradio as gr

dialog = "**Бот:** Привет! Что ты хочешь спросить?<br/>"

def run(txt):
    global dialog
    answer = chain.invoke(txt)
    dialog += f"**Я:** {txt}<br/>**Бот:** {answer}<br/>"
    return dialog

with gr.Blocks() as app:
    gr.Markdown('## Гарри Поттер Чат')
    out = gr.Markdown(dialog)
    inp = gr.Textbox(label='Твой вопрос')
    btn = gr.Button('Сказать')
    btn.click(fn=run,inputs=inp,outputs=out)

app.launch(share=True)